**Pandas Notebook 6**

A demonstration of advanced `pandas` syntax to accompany class Lecture.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

#More on `Groupby`





Try to predict the results of the `groupby` operation shown. The answer is below the image.
The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd".

In [ ]:
# Form a data frame using dictionary
# Answer Here
ds = pd.DataFrame(dict(x=[3, 1, 4, 1, 5, 9, 2, 5, 6],
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']),
                      index=list('ABCABCACB') )

In [ ]:
#Use groupby on index and get max of each group
ds.groupby(ds.index).agg(max)

<ipython-input-10-1f710b395442>:2: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ds.groupby(ds.index).agg(max)


,x,y
A,3,hi
B,6,tx
C,9,sd


### `groupby.size` and `groupby.count()`

In [ ]:
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'],
                   'num':[1, 2, 3, 4, np.NaN, 4],
                   'state':[np.NaN, 'tx', 'fl', 'hi', np.NaN, 'ak']})
df

,letter,num,state
0,A,1.0,NaN
1,A,2.0,tx
2,B,3.0,fl
3,C,4.0,hi
4,C,NaN,NaN
5,C,4.0,ak


`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [ ]:
# Use groupby with size()
df.groupby('letter').size()

,0
letter,
A,2
B,1
C,3


`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [ ]:
# Use groupby with count()
df.groupby('letter').count()

,num,state
letter,,
A,2,1
B,1,1
C,2,2


You might recall `value_counts()` function we talked about last week. What's the difference?

In [ ]:
# Use value_count() on DataFrame described above
df.groupby('letter').value_counts()

letter  num  state
A       2.0  tx       1
B       3.0  fl       1
C       4.0  ak       1
             hi       1
Name: count, dtype: int64

Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the resulting `Series` in descending order.

## Filtering by Group

In [5]:
# Let's read the elections dataset
elections = pd.read_csv("/content/drive/MyDrive/Data for Pandas/elections.csv")
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


Let's keep only the elections years where the maximum vote share `%` is less than 45%.

In [ ]:
# use filter function
f = lambda sf: sf['%'].sum() < 45
elections.groupby('%').filter(f)

,Year,Candidate,Party,Popular vote,Result,%
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583
7,1836,Hugh Lawson White,Whig,146109,loss,10.005985
...,...,...,...,...,...,...
174,2016,Evan McMullin,Independent,732273,loss,0.539546
175,2016,Gary Johnson,Libertarian,4489235,loss,3.307714
177,2016,Jill Stein,Green,1457226,loss,1.073699
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


### `groupby` Puzzle

Assume that we want to know the best election by each party.

#### Attempt #1

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [ ]:
# Use agg(max)
elections.groupby("Party").max().head(10)

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1976,Thomas J. Anderson,873053,loss,21.554001
American Independent,1976,Lester Maddox,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2016,Michael Peroutka,203091,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,2020,Woodrow Wilson,81268924,win,61.344703


#### Attempt #2

Next, we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above). Here's what the first rows of the correct output should look like:


In [ ]:
# Use sort and then groupby as describe in the lec
elections_sorted_by_percent = elections.sort_values("%", ascending=False)

In [ ]:
# grab first entry
elections_sorted_by_percent.groupby("Party").first()

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2008,Chuck Baldwin,199750,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,1964,Lyndon Johnson,43127041,win,61.344703


#### Alternative Solutions

You'll soon discover that with Pandas rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity, and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should, in general, try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here are a couple of other ways of obtaining the same result (in each case, we only show the top part with `head()`). The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by `Party` to match the requested formatting:

In [ ]:
# Use idxmax function
best_per_party = elections.loc[elections.groupby("Party")["%"].idxmax()]

Party
Democratic     17
Independent     9
Republican      3
Name: %, dtype: int64

In [ ]:
# This is the computational part, Extract DataFrame based on above mentioned condition

# Apply sort_index to match the formatting above
elections.sort_index(ascending=False)


,Year,Candidate,Party,Popular vote,Result,%
181,2020,Howard Hawkins,Green,405035,loss,0.255731
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979
179,2020,Donald Trump,Republican,74216154,loss,46.858542
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
177,2016,Jill Stein,Green,1457226,loss,1.073699
...,...,...,...,...,...,...
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878


Another approach is listed below. And this one doesn't even use `groupby`!

This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.

In [ ]:
# Formatting, Again, the 2nd line is purely formatting:
best_per_party2 = elections.sort_values("%").drop_duplicates(["Party"], keep="last")
best_per_party2

,Year,Candidate,Party,Popular vote,Result,%
148,1996,John Hagelin,Natural Law,113670,loss,0.118219
164,2008,Chuck Baldwin,Constitution,199750,loss,0.152398
110,1956,T. Coleman Andrews,States' Rights,107929,loss,0.174883
147,1996,Howard Phillips,Taxpayers,184656,loss,0.192045
136,1988,Lenora Fulani,New Alliance,217221,loss,0.237804
89,1932,William Z. Foster,Communist,103307,loss,0.261069
127,1980,Barry Commoner,Citizens,233052,loss,0.270182
50,1896,John M. Palmer,National Democratic,134645,loss,0.969566
78,1920,Parley P. Christensen,Farmer–Labor,265398,loss,0.995804
42,1888,Alson Streeter,Union Labor,146602,loss,1.288861


*Challenge:* See if you can find yet another approach that still gives the same answer.

### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [ ]:
grouped_by_party = elections.groupby("Party")
type(grouped_by_party)
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

The `key`s of the dictionary are the groups (in this case, `Party`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [42]:
# use get_group to extract 'Socialists' party results
grouped_elections = elections.groupby('Party')
socialists_results = grouped_elections.get_group('Socialist')
print(socialists_results)


     Year        Candidate      Party  Popular vote Result         %  \
58   1904   Eugene V. Debs  Socialist        402810   loss  2.985897   
62   1908   Eugene V. Debs  Socialist        420852   loss  2.850866   
66   1912   Eugene V. Debs  Socialist        901551   loss  6.004354   
71   1916  Allan L. Benson  Socialist        590524   loss  3.194193   
76   1920   Eugene V. Debs  Socialist        913693   loss  3.428282   
85   1928    Norman Thomas  Socialist        267478   loss  0.728623   
88   1932    Norman Thomas  Socialist        884885   loss  2.236211   
92   1936    Norman Thomas  Socialist        187910   loss  0.412876   
95   1940    Norman Thomas  Socialist        116599   loss  0.234237   
102  1948    Norman Thomas  Socialist        139569   loss  0.286312   

    first_name First_name  
58      Eugene     Eugene  
62      Eugene     Eugene  
66      Eugene     Eugene  
71       Allan      Allan  
76      Eugene     Eugene  
85      Norman     Norman  
88      Nor

## Pivot Tables

### `Groupby` with multiple columns

We want to build a table showing the total number of babies born of each sex in each year. One way is to `groupby` using both columns of interest:

In [21]:
# Find total count of baby names for both female and Male for each year
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()

pivot_table = babynames.pivot_table(
    index='Year',
    columns='Sex',
    aggfunc='sum',
    values='Count',
    fill_value=0
)
pivot_table



Sex,F,M
Year,,
1910,5950,3213
1911,6602,3381
1912,9804,8142
1913,11860,10234
1914,13815,13111
...,...,...
2019,184384,202941
2020,173966,189341
2021,174190,189016


### `pivot_table`

In [22]:
# Find total count of baby names for both female and Male for each year using Pivot table
pivot_table = babynames.pivot_table(
    index='Year',
    columns='Sex',
    values='Count',
    aggfunc='sum',
    fill_value=0
)
pivot_table

Sex,F,M
Year,,
1910,5950,3213
1911,6602,3381
1912,9804,8142
1913,11860,10234
1914,13815,13111
...,...,...
2019,184384,202941
2020,173966,189341
2021,174190,189016


### `pivot_table` with Multiple values

In [24]:
# Form a pivot table as describr in Lecture Slides
babynames_pivot = babynames.pivot_table(
index = "Year", # rows (turned into index)
columns = "Sex", # column values
values = ["Count", "Name"],
aggfunc = np.max, # group operation
)
babynames_pivot.head(6)

<ipython-input-24-d8ed072487c1>:2: FutureWarning: The provided callable <function max at 0x7a6bc350f880> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  babynames_pivot = babynames.pivot_table(


Count          Name         
Sex      F     M       F        M
Year                             
1910   295   237  Yvonne  William
1911   390   214   Zelma   Willis
1912   534   501  Yvonne  Woodrow
1913   584   614   Zelma   Yoshio
1914   773   769   Zelma   Yoshio
1915   998  1033    Zita    Yukio

## Join Tables

What if we want to know the popularity of presidential candidates' first names in California in 2022? What can we do?

In [35]:
elections.head(10)


,Year,Candidate,Party,Popular vote,Result,%,first_name,First_name
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew,Andrew
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John,John
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew,Andrew
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John,John
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew,Andrew
5,1832,Henry Clay,National Republican,484205,loss,37.603628,Henry,Henry
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583,William,William
7,1836,Hugh Lawson White,Whig,146109,loss,10.005985,Hugh,Hugh
8,1836,Martin Van Buren,Democratic,763291,win,52.272472,Martin,Martin
9,1836,William Henry Harrison,Whig,550816,loss,37.721543,William,William


In [36]:
# Collect baby names for 2022
babynames_ca_2022 = babynames[(babynames['State'] == 'CA') & (babynames['Year'] == 2022)]

In [37]:
# Use split the candidate names in elections dataframe
elections['First_name'] = elections['Candidate'].str.split().str[0]
california_candidates_2022 = elections[(elections['Year'] == 2022)]

`join` in pandas

In [38]:
#Merge both elections and babynames and report your analysis
merged_df = pd.merge(california_candidates_2022, babynames_ca_2022, left_on='First_name', right_on='Name')
popularity = merged_df.groupby('First_name')['Count'].sum().reset_index()
final_df = pd.merge(california_candidates_2022, popularity, on='First_name', how='left')

In [39]:
# Sort using Count
final_df[['Candidate', 'First_name', 'Count']]

,Candidate,First_name,Count
